In [1]:
import sys
import os
# Add the parent directory of the notebook (cxb1114) to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)  

import torch
import pytorch_lightning.callbacks.model_checkpoint
import pytorch_lightning.callbacks.early_stopping
torch.serialization.add_safe_globals([
    pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint,
    pytorch_lightning.callbacks.early_stopping.EarlyStopping
])
import torchaudio

from IPython.display import Audio, display

from Multi_Decoder_DPRNN.model import MultiDecoderDPRNN

from speechbrain.inference.separation import SepformerSeparation

from asteroid.models import BaseModel

KeyboardInterrupt: 

## Multi Decoder DPRNN

In [ ]:
file_path = r"C:\Users\conno\Desktop\UoB_Project\cxb1114\Multi_Decoder_DPRNN\2_mixture.wav"
mixture, sample_rate = torchaudio.load(file_path)
print("mixture audio(Input)")
display(ipd.Audio(data=mixture, rate=sample_rate))

what is n_src of below? Can it be changed? Or is this the complete model that detects n_src?

In [ ]:
model = MultiDecoderDPRNN.from_pretrained("JunzheJosephZhu/MultiDecoderDPRNN").eval()
if torch.cuda.is_available():
    model.cuda()
    mixture = mixture.cuda()
    

display(Audio(data=mixture, rate=sample_rate))
sources_est = model.separate(mixture)#.cpu()
for i, source in enumerate(sources_est):
  print(f"estimated source {i}")
  display(Audio(data=source, rate=sample_rate))

In [ ]:
file_path = r"C:\Users\conno\Desktop\UoB_Project\cxb1114\data\libricss\OV40\overlap_ratio_40.0_sil0.1_1.0_session0_actual39.5\record\utterances\utterance_6.wav"

# Load audio
mixture, sample_rate = torchaudio.load(file_path)


# If model expects mono, convert
if mixture.shape[0] > 1:
    mixture = mixture.mean(dim=0, keepdim=True)


# If model expects a specific sample rate, resample
target_sr = 8000 
if sample_rate != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
    mixture = resampler(mixture)
    sample_rate = target_sr
    
# Play processed audio
print("mixture audio (Input)")
display(Audio(data=mixture, rate=sample_rate))

# Move to GPU if available
if torch.cuda.is_available():
    model.cuda()
    mixture = mixture.cuda()


sources_est = model.separate(mixture).cpu()
for i, source in enumerate(sources_est):
    print(f"estimated source {i}")
    display(Audio(data=source, rate=sample_rate))

## Sepformer

In [ ]:
# Load the pretrained SepFormer from SpeechBrain (WSJ0‑2Mix)
model = SepformerSeparation.from_hparams(
    source="speechbrain/sepformer-wsj02mix",
    savedir="pretrained_models/sepformer-wsj02mix"
)

# Function to separate and play sources
def separate_and_play(file_path):
    print("Original audio:")
    display(Audio(file_path))  # Display the original audio file

    est_sources = model.separate_file(path=file_path)
    print("Separated sources shape:", est_sources.shape)
    sr = 8000  # model expects 8 kHz audio
    outputs = []
    for i in range(est_sources.shape[2]):
        speaker = est_sources[:, :, i].detach().cpu()
        out_path = f"source_{i+1}.wav"
        torchaudio.save(out_path, speaker, sr)
        outputs.append(Audio(out_path))
    for audio in outputs:
        display(audio)


In [ ]:
# Example usage
file_path = r"data\libricss\OV20\overlap_ratio_20.0_sil0.1_1.0_session1_actual20.5\record\utterances\utterance_5.wav"
separate_and_play(file_path)

## CSS

In [ ]:
# Load the pretrained TF-GridNet model (2-speaker)
model = BaseModel.from_pretrained("mpariente/TF-GridNet_Libri2Mix_sepclean")

def separate_file(file_path, output_dir="separated_outputs"):
    """
    Separates a mixed audio file into two sources using TF-GridNet.
    
    Args:
        file_path (str): Path to the input .wav file.
        output_dir (str): Directory to save the separated source files.
        
    Returns:
        List[str]: Paths of the saved separated source .wav files.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Run separation (reads file internally)
    est_sources = model.separate_file(path=file_path)  # shape: (1, n_srcs, time)
    est_sources = est_sources.squeeze(0).cpu()         # shape: (n_srcs, time)

    # Save each source
    sample_rate = 16000  # TF-GridNet trained on 16 kHz data
    output_paths = []
    for idx, source in enumerate(est_sources):
        out_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(file_path))[0]}_src{idx+1}.wav")
        torchaudio.save(out_path, source.unsqueeze(0), sample_rate)
        output_paths.append(out_path)

    return output_paths

# Example usage:
# file_path = "path/to/your/mixed_audio.wav"
# separated_paths = separate_file(file_path)
# print("Separated files:", separated_paths)
